# Imports and Setup

In [ ]:
import sys

sys.path.append("..")

# Import torchquad integrators and helper functions
from torchquad import Trapezoid, Simpson, Boole, MonteCarlo, VEGAS
from torchquad import set_precision, set_log_level, enable_cuda

# Import numerical backend functions
import torch
import numpy as np
import tensorflow as tf
import jax
import jax.numpy as jnp

# Imports for time measurement, plotting, etc.
from autoray import numpy as anp
from autoray import to_numpy
import matplotlib.pyplot as plt
import torch.utils.benchmark as benchmark
import time
from itertools import product

# Setup needed so that Torch works with CUDA and Tensorflow does not crash
enable_cuda()
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

use_double_precision = False
if use_double_precision:
    set_precision("double", backend="torch")
    set_precision("double", backend="jax")
else:
    set_precision("float", backend="torch")
    set_precision("float", backend="jax")

In [ ]:
# Show if CUDA is enabled for the backends
print(
    f"JAX: devices: {jax.devices()}, default device:"
    f" {jnp.array(1.0).device_buffer.device()}"
)
print(
    f"Tensorflow: built with CUDA: {tf.test.is_built_with_cuda()}, devices:"
    f" {tf.config.list_physical_devices('GPU')}"
)
print(
    f"Torch: CUDA enabled: {torch.cuda.is_initialized()}, "
    f"CUDA support and version: {torch.cuda.is_available()},"
    f" {torch.version.cuda}, cuDNN support and version: {torch.backends.cudnn.enabled},"
    f" {torch.backends.cudnn.version()}"
)

# Definition of Example Functions and Integration Domains


In [ ]:
def domain_to_backends(integration_domain):
    """
    Convert an integration domain given as list to tensors for each backend
    """
    return {
        "numpy": np.array(integration_domain),
        "torch": torch.tensor(integration_domain),
        "jax": jnp.array(integration_domain),
        "tensorflow": tf.constant(integration_domain),
    }


def gaussian(a, b, c, x):
    """
    Gaussian function, as defined at
    https://en.wikipedia.org/wiki/Gaussian_function
    a: vertical size
    b: centre position (dim)
    c: horizontal size
    x: points to be evaluated (N x dim)
    """
    off = x - b
    length_sqr = anp.sum(off * off, axis=1)
    return a * anp.exp(-length_sqr / (2.0 * c**2))


def gaussian_peaks(x):
    """An example integrand function which is costly to evaluate"""
    centres = [[0.0, 0.0], [0.5, 0.5], [0.6, 0.8], [1.0, 0.2]]
    vertical_sizes = [2.0, 0.3, 1.0, 1.0]
    horizontal_sizes = [0.1, 0.3, 0.05, 0.05]
    for k in range(21):
        cx = k / 20.0
        cy = cx
        centres.append([cx, cy])
        vertical_sizes.append(1.0 * (1.0 if k % 2 == 0 else -1.0))
        horizontal_sizes.append(0.05)
    values = []
    for a, b, c in zip(vertical_sizes, centres, horizontal_sizes):
        b = anp.array(b, like=x)
        values.append(gaussian(a, b, c, x))
    result = sum(values)
    return result


functions = {
    "sin_2d": {
        "funcs": {
            "numpy": lambda x: np.prod(np.sin(x), axis=1),
            "torch": lambda x: torch.prod(torch.sin(x), dim=1),
            "jax": lambda x: jnp.prod(jnp.sin(x), axis=1),
            "tensorflow": lambda x: tf.reduce_prod(tf.sin(x), axis=1),
        },
        "domains": domain_to_backends([[0.0, 1.0]] * 2),
        "dim": 2,
        "solution": (1.0 - np.cos(1.0)) ** 2,
    },
    "sin_5d": {
        "funcs": {
            "numpy": lambda x: np.prod(np.sin(x), axis=1),
            "torch": lambda x: torch.prod(torch.sin(x), dim=1),
            "jax": lambda x: jnp.prod(jnp.sin(x), axis=1),
            "tensorflow": lambda x: tf.reduce_prod(tf.sin(x), axis=1),
        },
        "domains": domain_to_backends([[0.0, 1.0]] * 5),
        "dim": 5,
        "solution": (1.0 - np.cos(1.0)) ** 5,
    },
    "exp_2d": {
        "funcs": {
            "numpy": lambda x: np.prod(np.exp(x), axis=1),
            "torch": lambda x: torch.prod(torch.exp(x), dim=1),
            "jax": lambda x: jnp.prod(jnp.exp(x), axis=1),
            "tensorflow": lambda x: tf.reduce_prod(tf.exp(x), axis=1),
        },
        "domains": domain_to_backends([[-10.0, 10.0]] * 2),
        "dim": 2,
        "solution": np.exp(20.0) - 2.0 + np.exp(-20.0),
    },
    "steps_extreme": {
        "funcs": {
            "numpy": lambda x: np.where(
                x[:, 0] < 0.0, 1e-17, np.where(x[:, 1] < 0.0, 1.0, -1.0)
            ),
            "torch": lambda x: torch.where(
                x[:, 0] < 0.0,
                torch.tensor(1e-17),
                torch.where(x[:, 1] < 0.0, torch.tensor(1.0), torch.tensor(-1.0)),
            ),
            "jax": lambda x: jnp.where(
                x[:, 0] < 0.0, 1e-17, jnp.where(x[:, 1] < 0.0, 1.0, -1.0)
            ),
            "tensorflow": lambda x: tf.where(
                x[:, 0] < 0.0, 1e-17, tf.where(x[:, 1] < 0.0, 1.0, -1.0)
            ),
        },
        "domains": domain_to_backends([[-1.0, 1.0]] * 2),
        "dim": 2,
        "solution": 2e-17,
    },
    "gaussian_peaks_2d": {
        "funcs": {
            "numpy": gaussian_peaks,
            "torch": gaussian_peaks,
            # "jax": jax.jit(gaussian_peaks),
            "jax": gaussian_peaks,
            "tensorflow": gaussian_peaks,
        },
        "domains": domain_to_backends([[0.0, 1.0]] * 2),
        "dim": 2,
        "solution": 0.1937366401593782,
    },
}


for backend, fun in functions["steps_extreme"]["funcs"].items():
    assert fun(anp.array([[-0.5, -0.5]], like=backend)) == 1e-17
    assert fun(anp.array([[0.5, -0.5]], like=backend)) == 1.0
    assert fun(anp.array([[0.5, 0.5]], like=backend)) == -1.0

Here I defined the following functions and corresponding integration domains:

* $sin\_2d(x) = \prod_{d=1}^{2} \sin(x_d)$  
  domain: $[0, 1]^2$
* $sin\_5d(x) = \prod_{d=1}^{5} \sin(x_d)$  
  domain: $[0, 1]^5$
* $exp\_2d(x) = \prod_{d=1}^{2} \exp(x_d)$  
  domain: $[-10, 10]^2$
* $steps\_extreme(x) = \begin{cases} \epsilon, && x_1 < 0 \\ 1 && x_1 \geq 0 \land x_2 < 0 \\ -1, && \text{ otherwise }\end{cases}$  
  domain: $[-1, 1]^2$
  
The 2D functions can be visualized:

In [ ]:
def plot_2d_integrand(func, domain, title, n_per_dim=30):
    """Plot a two-dimensional integrand function"""
    X, Y = np.meshgrid(
        np.linspace(domain[0][0], domain[0][1], n_per_dim),
        np.linspace(domain[1][0], domain[1][1], n_per_dim),
    )
    points = np.stack([X, Y], axis=2).reshape([-1, 2])
    values = func(points).reshape([n_per_dim, n_per_dim])
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection="3d")
    ax.plot_surface(X, Y, values, rstride=1, cstride=1)
    ax.set_title(title)
    plt.show()


def show_plots():
    for function_name in ["sin_2d", "exp_2d", "steps_extreme"]:
        function_data = functions[function_name]
        domain = function_data["domains"]["numpy"]
        func = function_data["funcs"]["numpy"]
        plot_2d_integrand(func, domain, function_name)


show_plots()

# Accuracy Comparisons

Since the integration rules are the same for all backends, the accuracies should only differ significantly if the backends use different floating point numbers or sum up elements in a different orders.

For the Newton Cotes quadrature rules, Torch orders the points differently than JAX and Numpy because of the [different meshgrid behaviour](https://github.com/pytorch/pytorch/issues/15301).

For the JAX and Torch backends, the precision can be set at the beginning of this notebook.

In [ ]:
def calculate_errors(function_name, integrator, N):
    """Calculate absolute and relative errors for the given function and parameters"""
    function_data = functions[function_name]
    errors = {}
    for backend, func in function_data["funcs"].items():
        result = integrator.integrate(
            func,
            dim=function_data["dim"],
            N=N,
            integration_domain=function_data["domains"][backend],
        )
        result = to_numpy(result)
        solution = function_data["solution"]
        errors[backend] = {
            "error_abs": np.abs(result - solution),
            "error_rel": np.abs((result - solution) / solution),
            "N": N,
        }
    return errors


def plot_errors(errors, title):
    """Show calculated errors in a bar plot"""
    backends = sorted(errors.keys())
    absolute_errors = [float(errors[backend]["error_abs"]) for backend in backends]
    print(f"Plotting bars {backends}, {absolute_errors}")

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.bar(backends, absolute_errors)
    ax.set_title(title)
    # ~ ax.set_yscale("log")
    ax.set_ylabel("absolute error")
    ax.grid(True, alpha=0.3)
    plt.show()

In [ ]:
def compare_accuracies():
    errors_sin_2d = calculate_errors("sin_2d", Boole(), 994009)
    errors_sin_5d = calculate_errors("sin_5d", Boole(), 13**5)
    errors_exp = calculate_errors("exp_2d", Boole(), 994009)
    errors_steps = calculate_errors("steps_extreme", Trapezoid(), 100**2)

    plot_errors(errors_sin_2d, "Boole errors for the 2D sin function")
    plot_errors(errors_sin_2d, "Boole errors for the 5D sin function")
    plot_errors(errors_exp, "Boole errors for the 2D exp function")
    plot_errors(errors_steps, "Trapezoid errors for the function with steps")


compare_accuracies()

# Runtime Comparisons

In [ ]:
def get_Ns(dim, max_N, integrator_name, base):
    """
    Calculate a list of valid number of points for the given integrator
    which can be used to compare runtimes.
    The number of points is roughly defined by the sequence (base ^ k)_k
    """
    Ns = []
    N_desired = 1.0
    for _ in range(1000):
        N = -1
        if integrator_name == "MonteCarlo":
            N = int(N_desired)
        elif integrator_name == "Boole":
            n_per_dim = int(N_desired ** (1.0 / dim) + 1e-8)
            if n_per_dim < 5:
                n_per_dim = 5
            new_n_per_dim = n_per_dim - ((n_per_dim - 1) % 4)
            N = new_n_per_dim**dim
        elif integrator_name == "Simpson":
            n_per_dim = int(N_desired ** (1.0 / dim) + 1e-8)
            if n_per_dim < 3:
                n_per_dim = 3
            new_n_per_dim = n_per_dim - ((n_per_dim - 1) % 2)
            N = new_n_per_dim**dim
        elif integrator_name == "Trapezoid":
            n_per_dim = int(N_desired ** (1.0 / dim) + 1e-8)
            if n_per_dim < 2:
                n_per_dim = 2
            N = n_per_dim**dim
        if N > max_N:
            break
        if (len(Ns) == 0 or Ns[-1] < N) and N >= 1:
            Ns.append(N)
        # Increase N roughly exponentially with the given base
        N_desired = N_desired * base
    return Ns


def compare_runtimes_torchbench(function_name, integrators, max_N):
    """Measure times with torch's benchmark function"""
    print(f"Comparing runtimes for {function_name}")
    set_log_level("SUCCESS")
    function_data = functions[function_name]
    measurements = []
    measurements_info = []
    for integrator, (backend, func) in product(
        integrators, function_data["funcs"].items()
    ):
        dim = function_data["dim"]
        domain = function_data["domains"][backend]
        integrator_name = type(integrator).__name__
        for N in get_Ns(dim, max_N, integrator_name, 10):
            print(f"integrator: {integrator_name}, N: {N}, backend: {backend}")
            measurements.append(
                benchmark.Timer(
                    stmt="to_numpy(integrator.integrate(func, dim=dim, N=N, integration_domain=domain))",
                    globals={
                        "to_numpy": to_numpy,
                        "integrator": integrator,
                        "func": func,
                        "dim": dim,
                        "N": N,
                        "domain": domain,
                    },
                    # num_treads probably only affects torch
                    num_threads=torch.get_num_threads(),
                    label=f"{function_name}, {integrator_name}",
                    sub_label=f"N: {N}",
                    description=backend,
                ).blocked_autorange(min_run_time=1)
            )
            measurements_info.append(
                {
                    "integrator_name": integrator_name,
                    "N": N,
                    "backend": backend,
                    "median": measurements[-1].median,
                }
            )
    set_log_level("INFO")
    compare = benchmark.Compare(measurements)
    compare.trim_significant_figures()
    compare.colorize()
    compare.print()
    return measurements, measurements_info


def compare_runtimes_direct(function_name, integrators, max_N):
    """Measure times directly with time.perf_counter"""
    print(f"Collecting runtimes for {function_name}")
    set_log_level("SUCCESS")
    function_data = functions[function_name]
    measurements_info = []
    for integrator, (backend, func) in product(
        integrators, function_data["funcs"].items()
    ):
        dim = function_data["dim"]
        domain = function_data["domains"][backend]

        def run_integration(N):
            return to_numpy(
                integrator.integrate(func, dim=dim, N=N, integration_domain=domain)
            )

        integrator_name = type(integrator).__name__
        for N in get_Ns(dim, max_N, integrator_name, 10):
            print(f"integrator: {integrator_name}, N: {N}, backend: {backend}")
            t0 = time.perf_counter()
            run_integration(N)
            t1 = time.perf_counter()
            run_integration(N)
            runtime2 = time.perf_counter() - t1
            runtime1 = t1 - t0
            measurements_info.append(
                {
                    "integrator_name": integrator_name,
                    "N": N,
                    "backend": backend,
                    "runtime1": runtime1,
                    "runtime2": runtime2,
                }
            )
    set_log_level("INFO")
    return measurements_info


measured_function_name = "gaussian_peaks_2d"
# measurements, measurements_info = compare_runtimes(
#    measured_function_name, [Trapezoid(), Boole(), MonteCarlo()], max_N=1000000
# )
measurements_info = compare_runtimes_direct(
    measured_function_name, [Trapezoid(), Boole(), MonteCarlo()], max_N=1000000
)

In [ ]:
def plot_lines(
    xs,
    ys,
    labels,
    title,
    ylabel,
    xlabel,
    xscale_log=False,
    yscale_log=False,
    output_file=None,
):
    """Create line plots"""
    fig, ax = plt.subplots()
    ax.set_title(title)

    for x, y, label in zip(xs, ys, labels):
        ax.plot(x, y, ".-", label=label)

    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    if xscale_log:
        ax.set_xscale("log")
    if yscale_log:
        ax.set_yscale("log")
    else:
        ax.set_ylim(bottom=0)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))

    if output_file is not None:
        fig.savefig(output_file, bbox_inches="tight")
    plt.show()


def plot_runtimes_torchbench(measurements_info, integrator_name):
    """Plot measurements collected with compare_runtimes_torchbench"""
    Ns = {}
    medians = {}
    for info in measurements_info:
        if info["integrator_name"] != integrator_name:
            continue
        backend = info["backend"]
        if backend not in Ns:
            Ns[backend] = []
            medians[backend] = []
        Ns[backend].append(info["N"])
        medians[backend].append(info["median"])
    xs = []
    ys = []
    labels = []
    for backend, Nvalues in Ns.items():
        xs.append(Nvalues)
        ys.append(medians[backend])
        labels.append(backend)
    plot_lines(
        xs,
        ys,
        labels,
        title=f"Runtimes for integrator {integrator_name}, integrand {measured_function_name}",
        xlabel="Number of evaluations",
        ylabel="Median time in s",
        xscale_log=True,
        yscale_log=True,
    )


def plot_runtimes_direct(measurements_info, integrator_name):
    """Plot measurements collected with compare_runtimes_direct"""
    Ns = {}
    runtimes = {}
    for info in measurements_info:
        if info["integrator_name"] != integrator_name:
            continue
        backend = info["backend"]
        if backend not in Ns:
            Ns[backend] = []
            runtimes[backend] = []
        Ns[backend].append(info["N"])
        runtimes[backend].append([info["runtime1"], info["runtime2"]])
    xs = []
    ys = []
    labels = []
    for backend, Nvalues in Ns.items():
        xs.append(Nvalues)
        ys.append([rt[0] for rt in runtimes[backend]])
        labels.append(f"{backend}, first run")
        xs.append(Nvalues)
        ys.append([rt[1] for rt in runtimes[backend]])
        labels.append(f"{backend}, second run")
    plot_lines(
        xs,
        ys,
        labels,
        title=f"Runtimes for integrator {integrator_name}, integrand {measured_function_name}",
        xlabel="Number of evaluations",
        ylabel="perf_counter time in s",
        xscale_log=True,
        yscale_log=True,
        output_file=f"tmp_{integrator_name}.pdf",
    )


def plot_all_runtimes():
    for integrator_name in ["MonteCarlo", "Trapezoid", "Boole"]:
        plot_runtimes_direct(measurements_info, integrator_name)


plot_all_runtimes()